#### PART 1. Obtenir les coordoonées geographiques des viles

In [1]:
# Importation des librairies
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

import plotly.express as px

import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
import os
import logging
import urllib

In [2]:
# liste donnée des villes
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
# création d'une fonction afin de recuper au format json les informations sur une ville, à partir de son nom passée en paramètre
# utlisation de l'API du site nominatim avec des critères en paramètre (format de sortie + nom ville)
# utilisation d'un try/except car certaines noms dans la liste renvoi une erreur. Dans ce cas nous passons l'erreur et affectons une valeur vide aux données recherchés
def city_gps(city_name):
    url_nominatim = "https://nominatim.openstreetmap.org/search"
    criteria_nominatim = {"format":"json", "city":city_name, "country":"France"}
    r = requests.get(url_nominatim, params=criteria_nominatim)
    r.json()
    try :
        city_lon = r.json()[0]["lon"]
        city_lat = r.json()[0]["lat"]
    except :
        city_lon = ""
        city_lat = ""
    return city_lon, city_lat

In [4]:
#création d'un dataframe à partir du nom
dataset_cities = pd.DataFrame({"City":cities})

# ajout de deux colonnes avec les coordonnées gps (latitude & longitude)
dataset_cities["Latitude"] = dataset_cities["City"].apply(lambda x: city_gps(x)[0])
dataset_cities["Longitude"] = dataset_cities["City"].apply(lambda x: city_gps(x)[1])

In [5]:
# verification du résultat du Dataframe
dataset_cities

,City,Latitude,Longitude
0,Mont Saint Michel,-1.511459954959514,48.6359541
1,St Malo,-2.0260409,48.649518
2,Bayeux,-0.7024738,49.2764624
3,Le Havre,0.1079732,49.4938975
4,Rouen,1.0939658,49.4404591
5,Paris,2.3483915,48.8534951
6,Amiens,2.2956951,49.8941708
7,Lille,3.0635282,50.6365654
8,Strasbourg,7.7507127,48.584614
9,Chateau du Haut Koenigsbourg,7.3454923,48.2495226


In [6]:
#ajout d'un id pour chaque ville, insertion de la colonne en 1ère position avec la méthode "insert" car sinon l'ajout se fait en fin de dataframe
dataset_cities.insert(0, "id", [i for i in range(1,36)])

In [7]:
# vérification des résultats
dataset_cities

,id,City,Latitude,Longitude
0,1,Mont Saint Michel,-1.511459954959514,48.6359541
1,2,St Malo,-2.0260409,48.649518
2,3,Bayeux,-0.7024738,49.2764624
3,4,Le Havre,0.1079732,49.4938975
4,5,Rouen,1.0939658,49.4404591
5,6,Paris,2.3483915,48.8534951
6,7,Amiens,2.2956951,49.8941708
7,8,Lille,3.0635282,50.6365654
8,9,Strasbourg,7.7507127,48.584614
9,10,Chateau du Haut Koenigsbourg,7.3454923,48.2495226


#### PART 2. Obteniur les données météologiques concernant les villes

Le site openweathermap renvoi à partir de coordonnées GPS un jeu de données trés etoffés qui comprend des données courantes, et des prévisions par minutes, heures, et jours.
Nous avons fait le choix de ne garder que les données concernant les prévisions journalières, qui sont sur 7 jours.
Les données journalières étant également nombreuses, nous n'avons également retenu que certaines d'entre elles :
- le date du jour en question
- la temperature (en degré celsius)
- le taux d'humidité (e n%)
- la vitesse du vent (en m/seconde)
- la pluie (en mm par jour)

In [11]:
# création d'une fonction qui renvoie des infos clés sur la météo prévisionnelle.
# la fonction à trois paramètres : les coordonnées gps (latitude/longitude) et le jour pour lequel nous souhaitons les données
# utlisation de l'API du site openweathermap (owm) avec des critères passés en paramètres : la clé de l'API, les coordonnées GPS, l'exclusion de certaines données (exclude) et le choix des unités (units)
# a noter également deux points :
# i)nous avons transformé la date qui n'est pas au bon format
# ii) et nous avons utilisé un try/except pour gerer l'absence de la donnée sur la pluie liée au fait qu'il ne pleut pas. Dans ce cas nous afffectons la valeur 0

def city_weather(lat, lon, day):
    url_owm = "https://api.openweathermap.org/data/3.0/onecall?"
    criteria_owm = {"lat":lat, "lon":lon, "appid":"1d7498fefc66d74c38e726701b5fd131", "exclude":"current,minutely,hourly", "units":"metric"}
    data_forecast = requests.get(url_owm, params = criteria_owm).json()
    day_date = data_forecast["daily"][day]["dt"]
    day_date = datetime.fromtimestamp(day_date).strftime("%x")
    day_temp = data_forecast["daily"][day]["temp"]["day"]
    day_humidity = data_forecast["daily"][day]["humidity"]
    day_wind = data_forecast["daily"][day]["wind_speed"]
    try :
        day_rain = data_forecast["daily"][day]["rain"]
    except :
        day_rain=0
    return [day_date, day_temp, day_humidity, day_wind, day_rain]

In [12]:
#création d'un dataframe concernant les prévisions météo à partir des coordonnées gps
# les colonnes correspndent aux données météo prévisionnelles
# les lignes correspondent aux villes et aux jours (donc plusieurs lignes pour une même ville)
dataset_forecast = pd.DataFrame(columns = ["id", "Date", "Temperature", "Humidity", "Wind", "Rain"])
dataset_forecast

,id,Date,Temperature,Humidity,Wind,Rain


In [13]:
#itération sur les villes (35 villes) et les prévisions journalières (7 jours) pour compléter le dataset
# nous aurions pu remplacer les chiffres par une valeur variable en fonction des résultats précédent (len par exemple)
# nous utilisons la fonction city_weather créée précédemment
# nous créons un dataframe temporaire pour y affecter les données renvoyées par la fonction
# nous concaténons ce dataframe temporaire au dataframe principal
for city in range(len(dataset_cities)):
    for i in range(7):
        response = city_weather(dataset_cities["Latitude"][city], dataset_cities["Longitude"][city], i)
        dataset_forecast_bis = pd.DataFrame({"id":city+1, "Date":response[0], "Temperature":response[1], "Humidity":response[2], "Wind":response[3], "Rain":response[4]}, index=[i+7*city])
        dataset_forecast = pd.concat([dataset_forecast, dataset_forecast_bis], ignore_index=True)

C:\Users\SD\AppData\Local\Temp\ipykernel_24628\4114757548.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_forecast = pd.concat([dataset_forecast, dataset_forecast_bis], ignore_index=True)


In [14]:
# la visualisation du dataframe permet de confirmer les résultats. nous avons 245 lignes soit 35 villes x 7 jours
dataset_forecast.shape

(245, 6)

In [15]:
# nous avons bien également 7 lignes pour le même id (ville)
dataset_forecast.head(40)

,id,Date,Temperature,Humidity,Wind,Rain
0,1,12/17/23,28.15,80,8.28,15.94
1,1,12/18/23,27.68,81,8.28,32.09
2,1,12/19/23,26.91,84,7.41,40.21
3,1,12/20/23,25.28,93,8.03,88.39
4,1,12/21/23,28.40,78,7.10,0.69
5,1,12/22/23,28.02,78,6.06,2.69
6,1,12/23/23,28.24,72,3.18,0.76
7,2,12/17/23,28.25,80,6.88,37.59
8,2,12/18/23,26.52,87,6.76,65.28
9,2,12/19/23,27.27,83,7.55,15.14


In [16]:
# nous vérifions également le format des données
dataset_forecast.dtypes

id              object
Date            object
Temperature    float64
Humidity        object
Wind           float64
Rain           float64
dtype: object

Sur le fond, nous nous sommes interrogés sur les données renvoyées et notamment la temperature qui sont censées être celles du mois de décembre !
Nous avons vérifié le json renvoyé par request est les données restituées sont bien celles obtenues du site.
En première lecture, il semblerait donc que ce soit les données obtenues du site qui sont incohérentes.
Par manque de temps nous n'avons pas pu investiguer plus en amont sur ce sujet.

In [17]:
# certains formats ne sont pas adaptés au type de données, ou dans un format qui prend de la place
# conversion des colonnes numériques avec la fonction pandas to_numeric avec en paramètre downcast ce que l'on souhaite (float)
# conversion de la colonne date avec la fonction pandas to_datetime
dataset_forecast[["Humidity", "Rain", "Temperature", "Wind"]] = dataset_forecast[["Humidity", "Rain", "Temperature", "Wind"]].apply(pd.to_numeric, downcast="float")
dataset_forecast["Date"] = dataset_forecast["Date"].apply(pd.to_datetime)

#verification du résultat
dataset_forecast.dtypes

id                     object
Date           datetime64[ns]
Temperature           float32
Humidity              float32
Wind                  float32
Rain                  float32
dtype: object

Nous avons à ce stade trop de données pour être capable de faire un choix sur une destination.
Nous allons pour chaque critère calculer une valeur unique sur l'ensemble de la période de prévision, pour chaque id (ville).
Nous avons retenu la médiane, plus robuste que la moyenne.

Nous considérerons donc pour la suite du travail que les donnée ssont justes.

In [18]:
# utlisation de la fonction groupby() à partir de l'id. le paramètre as_index permet de garder l'id comme une colonne sinon il passerait en index
median_forecast = dataset_forecast.groupby(["id"], as_index=False)[["Temperature", "Humidity", "Wind", "Rain"]].median()
median_forecast

,id,Temperature,Humidity,Wind,Rain
0,1,28.020000,80.0,7.41,15.94
1,2,28.240000,80.0,6.76,15.14
2,3,27.780001,80.0,7.79,13.89
3,4,27.840000,80.0,8.56,9.02
4,5,27.620001,79.0,9.81,6.77
5,6,27.450001,80.0,10.30,0.64
6,7,27.330000,81.0,10.10,2.35
7,8,26.940001,82.0,10.30,0.65
8,9,29.910000,33.0,9.41,0.00
9,10,32.610001,27.0,9.05,0.00


Maintenant que nous avons pour chaque ville (id) une valeur unique par paramètre, nous allons rajouter à notre dataframe initiale les colonnes concernant la météo previsionnelle

In [19]:
# utilisation de la fonction merge() pour fusionner les 2 dataframes en jointant sur la colonne commune "id"
dataset_cities = dataset_cities.merge(median_forecast, on=["id"])
dataset_cities

,id,City,Latitude,Longitude,Temperature,Humidity,Wind,Rain
0,1,Mont Saint Michel,-1.511460,48.635954,28.020000,80.0,7.41,15.94
1,2,St Malo,-2.026041,48.649518,28.240000,80.0,6.76,15.14
2,3,Bayeux,-0.702474,49.276462,27.780001,80.0,7.79,13.89
3,4,Le Havre,0.107973,49.493898,27.840000,80.0,8.56,9.02
4,5,Rouen,1.093966,49.440459,27.620001,79.0,9.81,6.77
5,6,Paris,2.348391,48.853495,27.450001,80.0,10.30,0.64
6,7,Amiens,2.295695,49.894171,27.330000,81.0,10.10,2.35
7,8,Lille,3.063528,50.636565,26.940001,82.0,10.30,0.65
8,9,Strasbourg,7.750713,48.584614,29.910000,33.0,9.41,0.00
9,10,Chateau du Haut Koenigsbourg,7.345492,48.249523,32.610001,27.0,9.05,0.00


In [20]:
# sauvgarde du résultat sous forme de fichier csv, sans les index
dataset_cities.to_csv("dataset_cities.csv", index=False)

Selon nos critères (arbitraire pour les besoins de l'exercice), les destinations les plus intéressantes seront celles avec :
- une quantité de pluie < 0.2
- une température moyenne supérieure à 25 degrés
- une humidité inférieure ou égale à 30%
- un vent au maximum de 10 m/sec

In [21]:
dataset_cities_results= dataset_cities[(dataset_cities.Rain < 0.2) & (dataset_cities.Humidity <= 30) & (dataset_cities.Wind <= 10) & (dataset_cities.Temperature > 25)]
dataset_cities_results

,id,City,Latitude,Longitude,Temperature,Humidity,Wind,Rain
9,10,Chateau du Haut Koenigsbourg,7.345492,48.249523,32.610001,27.0,9.05,0.0
10,11,Colmar,7.357964,48.077752,32.369999,27.0,9.05,0.0
11,12,Eguisheim,7.307962,48.044797,32.540001,27.0,9.05,0.0
27,28,Collioure,3.083155,42.525050,34.599998,29.0,7.27,0.0


In [22]:
# conversion des données Latitude/Longitude en nb décimal pour pouvoir être utilisées dans la carte
dataset_cities_results["Latitude"]=dataset_cities_results["Latitude"].astype(float, errors="raise")
dataset_cities_results["Longitude"]=dataset_cities_results["Longitude"].astype(float, errors="raise")

C:\Users\SD\AppData\Local\Temp\ipykernel_24628\1674794291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cities_results["Latitude"]=dataset_cities_results["Latitude"].astype(float, errors="raise")
C:\Users\SD\AppData\Local\Temp\ipykernel_24628\1674794291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cities_results["Longitude"]=dataset_cities_results["Longitude"].astype(float, errors="raise")


In [23]:
# création du graphe
# il semblerait qu'il y ai une inversion entre la latitude et la longitude, selon la source. nous avons donc passé la latitude de notre dataset en longitude pour la carte, et inversement
cities_results = px.scatter_mapbox(
    dataset_cities_results,
    title = "Localisation des villes à privilégier pour vos vacances",
    lat="Longitude",
    lon="Latitude",
    color="Temperature",
    mapbox_style="open-street-map",
    text="City",
    zoom=5,
)
cities_results.update_layout(width=1000,height = 900, margin={"r":0,"t":50,"l":0,"b":0}, title_x = 0.5, title_y = 0.98)
cities_results.update_traces(marker={'size': 15}) # permet de definir une taille fixe pour les points sur la carte
cities_results.show()

#### PART 3. Obtenir une liste d'hotels disponibles pour les destinations retenues à partir de booking.com

In [ ]:
class booking_spider(scrapy.Spider):
    name = "booking"
    base_lien = []

    def start_requests(self):
        for city in ["Collioure","Colmar"] :
            parameters_booking = {"ss":city, "no_rooms":"1", "checkin":"2023-12-18", "checkout":"2023-12-19", "group_adults":"1", "group_children":"0"}
            final_url =f'{"https://www.booking.com/searchresults.fr.html?"}{urllib.parse.urlencode(parameters_booking)}'
            yield scrapy.Request(url=final_url, callback=self.parse)

    def parse(self, response):
        links = response.xpath('//h3[@class="aab71f8e4e"]/a/@href').getall()
        for lien in links :
            lien = str(lien).split("?")[0]
            self.base_lien.append(lien)
            yield scrapy.Request(url=lien, callback=self.parse_detail)

    def parse_detail(self, response) :
        yield {
            'Name' : response.xpath('//h2[@class="d2fee87262 pp-header__title"]/text()').get(),
            'rating' : response.xpath('//div[@class="b5cd09854e d10a6220b4"]/text()').get(),
            'gps' : response.xpath('//a[@id="hotel_address"]/@data-atlas-latlng').get(""),
            'url' : response.request.url,
            'Comment' : response.xpath('//p[@class="a53cbfa6de b3efd73f69"]/text()').get()
        }
            
filename = "booking.json"

if filename in os.listdir("C:/Users/SD\Desktop/Cours/Jedha/Fullstack_Datascience/4 - Data_Collection_and_Management"):
        os.remove("C:/Users/SD\Desktop/Cours/Jedha/Fullstack_Datascience/4 - Data_Collection_and_Management" + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0',
    'LOG_LEVEL': logging.INFO,
    'FEED_EXPORT_ENCODING' : "utf-8", # résoud le pb de caractères
    "FEEDS": {filename : {"format": "json"},
    }
})

process.crawl(booking_spider)
process.start()


In [ ]:
# sauvegarde des fichiers sur S3
# les données de login ont été anonymisées
import boto3
session = boto3.Session(aws_access_key_id="xxxxxxxxxxxxxxxxxxxxx", aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

# créer une session "ressource"
s3 = session.resource("s3")

# créer un bucket sur AWS-S3
bucket = s3.create_bucket(Bucket = "projet-kayak", CreateBucketConfiguration={'LocationConstraint':'eu-west-3'})

# mettre les fichiers dans le bucket créé sur S3
bucket.put_object(Body="dataset_cities.csv", Key="dataset_cities.csv")
bucket.put_object(Body="booking.json", Key="booking.json")

#### PART 4. Pour aller plus loin

Avec plus de temps, nous aurions souhaiter notamment :
- mieux documenter les fonctions (descriptif des variables, objectif de la fonction)
- vérifier les données obtenues du site openweather
- nettoyer les données scrapées du site booking.com, et les présenter sous un format plus adaptées
- filtrer le nombre d'hôtels renvoyés afin d'en limiter le nombre, à partir d'un ou plusieurs critères (par ex top 5 rating ou prix)
- créer un dashboard avec streamlit par exemple afin de présenter l'ensemble du travail